In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2 as cv
import os
import seaborn as sns
import pandas as pd
from skimage.filters import sobel
from skimage.feature import greycomatrix, greycoprops
from skimage.measure import shannon_entropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from PIL import Image
from tqdm.notebook import tqdm

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

In [ ]:
def clahe(image, denoise=False, verbose=False, limit=None):
    # bgr = cv2.imread(image_path)
    bgr = image

    lab = cv.cvtColor(bgr, cv.COLOR_BGR2LAB)
    lab_planes = cv.split(lab)
    clahe = cv.createCLAHE(clipLimit=limit)
    lab_planes[0] = clahe.apply(lab_planes[0])
    #lab_planes[1] = clahe.apply(lab_planes[1])
    #lab_planes[2] = clahe.apply(lab_planes[2])
    lab = cv.merge(lab_planes)
    bgr = cv.cvtColor(lab, cv.COLOR_LAB2BGR)

    if denoise:
        bgr = cv.fastNlMeansDenoisingColored(bgr, None, 10, 10, 1, 3)
        #bgr = cv2.bilateralFilter(bgr, 5, 1, 1)

    if verbose:
        cv.imshow("test", bgr)
        cv.waitKey(0)
        cv.destroyAllWindows()
    return bgr

In [ ]:
SIZE = 256

In [ ]:
image_data = []
labels_data = []
image_paths = []

for directory_path in glob.glob("dataset/*"): 
    label = directory_path.split("\\")[-1]
    print(label)
    count = 0
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        im = Image.open(img_path).convert("RGB")
        new_img = im.resize((SIZE,SIZE),Image.BICUBIC)
        image = np.array(new_img)
        image = image[:, :, ::-1].copy()
        (h1, w1) = image.shape[:2]
        print(h1, w1)
        cv.imwrite(f"Data/progress/resized/resize_fish_{str(count)}_{str(label)}.jpg", image)
        image = clahe(image, denoise=True, limit=0)
        cv.imwrite(f"Data/progress/clahe/clahe_fish_{str(count)}_{str(label)}.jpg", image)
        image = cv.cvtColor(image, cv.COLOR_RGB2Lab)
        cv.imwrite(f"Data/progress/cielab/cielab_fish_{str(count)}_{str(label)}.jpg", image)
        image = image.reshape((image.shape[0] * image.shape[1], 3))
        clt = KMeans(n_clusters = 5)
        labels = clt.fit_predict(image)
        quant = clt.cluster_centers_.astype("uint8")[labels]
        quant = quant.reshape((h1, w1, 3))
        image = image.reshape((h1, w1, 3))
        cv.imwrite(f"Data/progress/kmeans/kmeans_fish_{str(count)}_{str(label)}.jpg", quant)
        cv.imwrite(f"Data/progress/segmented/segmented_fish_{str(count)}_{str(label)}.jpg", image)
        quant = cv.cvtColor(quant, cv.COLOR_LAB2BGR)
        image_gray = cv.cvtColor(quant, cv.COLOR_BGR2GRAY)
        cv.imwrite(f"Data/progress/grayscale/gray{str(count)}_{str(label)}.jpg",image_gray)
        image_data.append(image_gray)
        labels_data.append(label)
        image_paths.append(img_path)
        count = count + 1

In [ ]:
cv.imshow("image",image_data[55])
cv.waitKey(200)
cv.destroyAllWindows()

In [ ]:
image_data = np.array(image_data)
labels_data = np.array(labels_data)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(labels_data)
labels_encoded = le.transform(labels_data)

In [ ]:
def feature_extractor(dataset):
    image_dataset = pd.DataFrame()
    for image in range(dataset.shape[0]):
        #print(image)
        
        df = pd.DataFrame()
        
        img = dataset[image, :,:]
     
        #Full image
        #GLCM = greycomatrix(img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])
        GLCM = greycomatrix(img, [3], [0])       
        GLCM_Energy = greycoprops(GLCM, 'energy')[0]
        df['Energy_0'] = GLCM_Energy
        GLCM_corr = greycoprops(GLCM, 'correlation')[0]
        df['Corr_0'] = GLCM_corr
        GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
        df['Homogen_0'] = GLCM_hom       
        GLCM_contr = greycoprops(GLCM, 'contrast')[0]
        df['Contrast_0'] = GLCM_contr
        GLCM_Asm = greycoprops(GLCM, 'ASM')[0]
        df['ASM_0'] = GLCM_Asm


        GLCM2 = greycomatrix(img, [3], [np.pi/4])       
        GLCM_Energy2 = greycoprops(GLCM2, 'energy')[0]
        df['Energy_45'] = GLCM_Energy2
        GLCM_corr2 = greycoprops(GLCM2, 'correlation')[0]
        df['Corr_45'] = GLCM_corr2
        GLCM_hom2 = greycoprops(GLCM2, 'homogeneity')[0]
        df['Homogen_45'] = GLCM_hom2       
        GLCM_contr2 = greycoprops(GLCM2, 'contrast')[0]
        df['Contrast_45'] = GLCM_contr2
        GLCM_Asm2 = greycoprops(GLCM, 'ASM')[0]
        df['ASM_45'] = GLCM_Asm2

        GLCM3 = greycomatrix(img, [3], [np.pi/2])       
        GLCM_Energy3 = greycoprops(GLCM3, 'energy')[0]
        df['Energy_90'] = GLCM_Energy3
        GLCM_corr3 = greycoprops(GLCM3, 'correlation')[0]
        df['Corr_90'] = GLCM_corr3
        GLCM_hom3 = greycoprops(GLCM3, 'homogeneity')[0]
        df['Homogen_90'] = GLCM_hom3       
        GLCM_contr3 = greycoprops(GLCM3, 'contrast')[0]
        df['Contrast_90'] = GLCM_contr3
        GLCM_Asm3 = greycoprops(GLCM, 'ASM')[0]
        df['ASM_90'] = GLCM_Asm3

        GLCM4 = greycomatrix(img, [3], [3*np.pi/4])       
        GLCM_Energy4 = greycoprops(GLCM4, 'energy')[0]
        df['Energy_135'] = GLCM_Energy4
        GLCM_corr4 = greycoprops(GLCM4, 'correlation')[0]
        df['Corr_135'] = GLCM_corr4
        GLCM_hom4 = greycoprops(GLCM4, 'homogeneity')[0]
        df['Homogen_135'] = GLCM_hom4       
        GLCM_contr4 = greycoprops(GLCM4, 'contrast')[0]
        df['Contrast_135'] = GLCM_contr4
        GLCM_Asm4 = greycoprops(GLCM, 'ASM')[0]
        df['ASM_135'] = GLCM_Asm4
        
        

        image_dataset = image_dataset.append(df)
        
    return image_dataset

In [ ]:
image_features = feature_extractor(image_data)
column_list = image_features.columns

In [ ]:
image_features['output'] = labels_encoded

In [ ]:
image_features['filenames'] = image_paths

In [ ]:
image_features.to_csv("features.csv")
image_features.head()